In [0]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/Intro to AI Project/data/"
MODEL_PATH = "/content/drive/My Drive/Intro to AI Project/model/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Intro to AI Project/code/')
import pandas as pd
import numpy as np
import time
import pickle
from tqdm import *
from scipy.sparse import csr_matrix
import time
from utils.vocab import Vocabulary
import statistics

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pickle.load(open(DATA_PATH + 'recipe1m_train.pkl', 'rb'))
data = [{"id":x["id"],'ingredients':x['ingredients']} for x in data]

In [0]:
print(len(data))
length = [len(data[0]['ingredients'])]
for i in range (1, len(data)):
  length.append(len(data[i]['ingredients']))
print(statistics.mean(length))
print(statistics.mode(length))

639740
8.616803701503736
7


In [0]:
ingrs_vocab = pickle.load(open(DATA_PATH + 'recipe1m_vocab_ingrs.pkl', 'rb'))

In [0]:
sim = pd.DataFrame(columns = range(1,len(ingrs_vocab.idx2word.keys())),index=range(1,len(ingrs_vocab.idx2word.keys())))
sim = sim.fillna(0)
for i, entry in tqdm(enumerate(data)):
    ingrs_idx = list(set([ingrs_vocab.word2idx[x] for x in entry["ingredients"] if x in ingrs_vocab.word2idx]))
    for pos1 in range(0, len(ingrs_idx)-1):
          for pos2 in range(pos1+1, len(ingrs_idx)):
                i = ingrs_idx[pos1]
                j = ingrs_idx[pos2]
                sim.loc[i, j] += 1
                sim.loc[j, i] += 1

In [0]:
sim.to_pickle(MODEL_PATH+'cf_data_%d.pkl'%int(time.time()))

In [0]:
sim = pickle.load(open(MODEL_PATH + 'cf_data_1573361724.pkl', 'rb'))

In [0]:
sim

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,2570,2571,2572,2573,2574,2575,2576,2577,2578,2579,2580,2581,2582,2583,2584,2585,2586,2587,2588,2589,2590,2591,2592,2593,2594,2595,2596,2597,2598,2599,2600,2601,2602,2603,2604,2605,2606,2607,2608,2609
1,0,2943,553,2712,32,542,266,1892,69,1043,2196,90,768,283,0,668,0,0,67,0,721,456,239,874,63,32,13,15,277,0,0,309,4,0,1388,904,41,3,1,664,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2943,0,4904,74089,363,10200,8719,52688,2666,28336,58938,2260,48084,14960,1199,16912,2051,187,5047,1009,11508,16308,14108,25821,1945,1737,1688,3096,15961,34,74,24137,1833,1006,44082,23806,4614,2886,4609,39609,...,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,553,4904,0,18238,120,3889,3049,11971,858,6020,21010,667,8993,2040,210,6641,18,6,279,69,2295,3194,3827,5508,759,403,415,267,4773,3,8,3145,47,5,5735,3581,303,54,172,3420,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2712,74089,18238,0,507,14727,37465,126541,6262,52433,131533,6679,122311,22265,173,38539,25,295,2283,826,17469,37397,40703,51042,6345,2252,4041,3553,31287,142,90,33460,542,73,54991,30632,5411,963,2489,38560,...,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,32,363,120,507,0,183,110,354,18,95,456,17,220,26,7,107,0,0,25,0,24,69,103,129,10,6,4,6,128,0,0,99,0,2,165,96,4,3,9,128,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2606,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2607,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2608,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
lst = [ ] 
n = int(input("Enter number of elements : ")) 
  
for i in range(0, n): 
    ele = input() 
    lst.append(ele) 
      
print(lst) 

Enter number of elements : 7
chicken
potato
egg
carrot
bean
corn
eggplant
['chicken', 'potato', 'egg', 'carrot', 'bean', 'corn', 'eggplant']


In [0]:
lst_index = []
for i in range(0,len(lst)):
  index = ingrs_vocab.word2idx[lst[i]] 
  lst_index.append(index)
ingre_matrix = sim[lst_index].loc[lst_index]


In [0]:
def winner(list1):
    lst1 = [] 
    length = len(list1) 
    list1.sort() 
    lst1.append(list1[length-1])
    lst1.append(list1[length-2])
    return lst1
def get_winner(lst_index,ingre_matrix):
  win_ingre = []
  for i in range(0,len(lst_index)):
    win_ingre.append(winner(ingre_matrix.iloc[i].values.tolist()))
  win_index = []
  for i in range(0,len(lst_index)):
    l=[]
    win_index.append(l)
    for j in range(0, len(win_ingre[0])):
      l.append(ingre_matrix.iloc[[i]].columns[(ingre_matrix.iloc[[i]] == win_ingre[i][j]).iloc[0]][0])
  for i in range(0,len(lst_index)):
    win_index[i].append(sum(win_ingre[i]))
  win_index = pd.DataFrame(win_index)
  win_index[3] = lst_index
  win_index_lst = win_index.loc[win_index[2] == max(win_index[2])][[0,1,3]].values.flatten().tolist()
  return win_index_lst  

In [0]:
recommend_index = get_winner(lst_index,ingre_matrix)
for i in range(0,3):
  if (sim.loc[get_winner(lst_index,ingre_matrix)[i]].nlargest(3)).reset_index()['index'].values.tolist()[0] not in recommend_index:
    recommend_index.append((sim.loc[get_winner(lst_index,ingre_matrix)[i]].nlargest(3)).reset_index()['index'].values.tolist()[0])
  if (sim.loc[get_winner(lst_index,ingre_matrix)[i]].nlargest(3)).reset_index()['index'].values.tolist()[1] not in recommend_index:
    recommend_index.append((sim.loc[get_winner(lst_index,ingre_matrix)[i]].nlargest(3)).reset_index()['index'].values.tolist()[1])
  if (sim.loc[get_winner(lst_index,ingre_matrix)[i]].nlargest(3)).reset_index()['index'].values.tolist()[2] not in recommend_index:
    recommend_index.append((sim.loc[get_winner(lst_index,ingre_matrix)[i]].nlargest(3)).reset_index()['index'].values.tolist()[2])
recommend_ingre = pd.Series(recommend_index).map({k: v[0] for k,v in ingrs_vocab.idx2word.items()}).values.tolist()
recommend_ingre


['carrot', 'egg', 'potato', 'onion', 'pepper', 'oil', 'sugar', 'salt', 'flour']